<a href="https://colab.research.google.com/github/thabetAljbreen/RepoT5/blob/main/Sequence_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a Sequence Classification Model Exam

In this exam, you will be tasked with performing dataset preprocessing and fine-tuning a model for sequence classification. Complete each step carefully according to the instructions provided.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `aubmindlab/bert-base-arabertv02` for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/sanad_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

*Load* the dataset and split it into training and test sets. Use 20% of the data for testing.

In [1]:
pip install farasapy

In [2]:
!pip install arabert

In [4]:
from arabert.preprocess import ArabertPreprocessor

model_name="aubmindlab/bert-base-arabertv02"
arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري"
arabert_prep.preprocess(text)

'ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري'

In [3]:
!pip install datasets

In [8]:
from datasets import load_dataset

dataset = load_dataset('CUTD/sanad_df')
dataset = dataset['train'].train_test_split(test_size=0.2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 3000
    })
})

In [31]:
import pandas as pd
pd.Series(dataset["train"]["label"]).unique()

array(['Culture', 'Finance', 'Medical'], dtype=object)

## Step 2: Clean Unnecessary Columns

Remove any columns from the dataset that are not needed for training.

In [9]:
print(dataset['train'].column_names)

['Unnamed: 0', 'text', 'label']


In [10]:
dataset.remove_columns(['Unnamed: 0'])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3000
    })
})

## Step 3: Splitting the Dataset

Split the dataset into training and testing sets, ensuring that 20% of the data is used for testing.

In [ ]:
#already done dataset = dataset['train'].train_test_split(test_size=0.2)

## Step 4: Tokenizing the Data

Initialize a tokenizer for the model.

In [11]:
!pip install transformers==4.31.0

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Step 5: Preprocessing the Text

Map the tokenization function to the dataset. Ensure the text data is processed using truncation to handle sequences that exceed the model's input size. Please do any further preprocessing.

**Bonus**: If you performed more comprehensive preprocessing, such as removing links, converting text to lowercase, or applying additional preprocessing techniques.

In [13]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

### Step 6: Label Encoding

Convert the categorical labels into numerical format using a label encoder if needed.

In [14]:
# labels are finance etc are not numerical

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(dataset['train']['label'])

def label_encode(encoding):
  encoding['label'] = le.transform(encoding['label'])
  return encoding

encoded_dataset = dataset.map(label_encode, batched=True)


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [15]:
encoded_dataset = tokenizer(dataset['train']['text'], return_tensors='pt', padding=True, truncation=True)


### Step 7: Data Collation for Padding

Prepare the data for training by ensuring all sequences in a batch are padded to the same length. Use a data collator to handle dynamic padding.

In [16]:
# aready did it encoded_dataset = tokenizer(dataset['train']['text'], return_tensors='pt', padding=True, truncation=True)



### Step 8: Model Initialization

Initialize a sequence classification model using the BERT-based architecture. Set the the right amount of output labels.

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("aubmindlab/bert-base-arabertv02", num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step 9: Training Arguments

Define the training arguments, including parameters such as
learning rate,
batch size,
 number of epochs,
 and weight decay.

In [41]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./thabet",
    learning_rate=2e-5,
    per_device_train_batch_size=20,
    num_train_epochs=3,
    weight_decay=0.01,
)

## Step 10: Trainer Initialization and Training

Set up the trainer with the model, training arguments, dataset, tokenizer, and data collator. Train the model using the dataset you processed earlier.

In [46]:
encoded_dataset=dict(encoded_dataset)

In [1]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    tokenizer=tokenizer,
)

trainer.train()

NameError: name 'model' is not defined

In [21]:
print(encoded_dataset.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
trai




















## Step 11: Inference

Once the model is trained, perform inference on a sample text to evaluate the model's prediction capabilities. Use the tokenizer to process the text, and then feed it into the model to get the predicted label.